# Advent of Code

## 2021-012-024
## 2021 024

https://adventofcode.com/2021/day/24

In [1]:
def parse_blocks(lines):
    # Each block is 18 lines, starting with "inp w"
    blocks = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("inp w"):
            blocks.append(lines[i:i+18])
            i += 18
        else:
            i += 1
    return blocks

def extract_parameters(block):
    # Given a single 18-line block, extract:
    # div z X  -> X is line 4
    # add x X  -> X is line 5 after mod x 26
    # add y Y  -> Y is line 15
    divz = int(block[4].split()[-1])
    checkx = int(block[5].split()[-1])
    addy = int(block[15].split()[-1])
    return divz, checkx, addy

def solve_monad(parameters):
    # parameters: list of (divz, checkx, addy) for each of the 14 blocks
    stack = []
    relations = [None] * 14

    # When divz == 1, we push (index, addy).
    # When divz == 26, we pop and relate indices.
    for i, (divz, checkx, addy) in enumerate(parameters):
        if divz == 1:
            # push
            stack.append((i, addy))
        else:
            # pop
            j, prev_addy = stack.pop()
            # The constraint: digits[i] = digits[j] + prev_addy + checkx
            # We want to find digits that are in [1..9].
            # Actually, to find them easily, we store this relation.
            relations[i] = (j, prev_addy + checkx)

    # Now we have a set of relations.
    # For each pair (i, (j, offset)), means digit[i] should equal digit[j] + offset.
    # We want the largest valid number: try to assign digits to maximize overall number.

    digits = [0]*14

    for i, rel in enumerate(relations):
        if rel is None:
            # This is a pushing block that didn't get paired yet 
            # (should not happen if input is correct, but let's just skip)
            continue
        j, offset = rel

        # If offset > 0, we want to reduce j-digit if possible to keep i-digit ≤ 9.
        # If offset < 0, we want to increase j-digit if possible to keep i-digit ≤ 9.

        # Let's pick the largest possible digits[i] and digits[j].
        # digits[i] = digits[j] + offset
        # To maximize the number, we start from the highest for digits[j].
        # We'll choose digits[j] as high as possible (9) and then digits[i] = 9 + offset.
        # If that doesn't fit (1 <= digits[i] <= 9), we adjust.
        
        # We'll do a simple approach:
        # We'll solve pairs after we've ensured that each pair is handled once.
        # The pairs appear when divz=26, we handle them in order.

        # If offset > 0: digits[j] + offset = digits[i], to get max number,
        # set digits[j] = 9 - offset (so digits[i] = 9)
        # If offset < 0: digits[i] = digits[j] + offset; to get max number,
        # set digits[j] = 9 (so digits[i] = 9 + offset), 
        # if digits[i] < 1, reduce digits[j] until digits[i]>=1.

        if offset > 0:
            # Want digits[i] = digits[j] + offset
            # Let's try to make digits[i] = 9
            # Then digits[j] = 9 - offset
            dj = 9 - offset
            di = 9
            # If dj < 1, we must shift down di
            if dj < 1:
                # Increase dj up to 1, and reduce di accordingly.
                dj = 1
                di = 1 + offset
        else:
            # offset <= 0
            # digits[i] = digits[j] + offset
            # Try digits[j] = 9
            # digits[i] = 9 + offset
            dj = 9
            di = 9 + offset
            if di > 9: 
                # If di > 9, we must reduce dj
                # so that di ≤ 9. di = dj + offset => dj = di - offset
                # If offset is negative, to reduce di, we must reduce dj.
                # Just try dj to make di=9:
                di = 9
                dj = di - offset
            if di < 1:
                # If still di < 1, we must lower dj to raise di.
                # Actually raising di not possible, must ensure di≥1:
                # di = dj + offset => dj = di - offset
                # If we set di=1, dj=1-offset
                di = 1
                dj = 1 - offset
                if dj>9:
                    # If this happens, it means we can't find a digit assignment easily.
                    # But given the puzzle structure, there's always a solution.
                    # Let's trust the logic that we can find a match.
                    # We'll just clamp to valid range. (Should not occur with correct input.)
                    dj = 9
                    di = dj + offset
                    # Ensure validity again.
                    if not (1 <= di <= 9):
                        raise ValueError("No valid assignment found, check logic.")

        # Ensure both are within [1..9]
        if not(1<=di<=9 and 1<=dj<=9):
            # Adjust strategy if needed
            # We'll just try a minimal difference:
            # Instead of forcing one side max, let's try a balanced approach.
            # We know offset = di - dj. We must find 1<=di,dj<=9 s.t di - dj = offset.
            # Start dj from 1 to 9 and check if di fits.
            found = False
            for candidate_dj in range(9,0,-1):
                candidate_di = candidate_dj + offset
                if 1<=candidate_di<=9:
                    di = candidate_di
                    dj = candidate_dj
                    found = True
                    break
            if not found:
                raise ValueError("No valid digits found for relation")

        digits[i] = di
        digits[j] = dj

    # For any digit not set (from the push blocks without direct constraints), set it high (9).
    for idx, d in enumerate(digits):
        if d == 0:
            digits[idx] = 9

    return digits

if __name__ == "__main__":
    with open("input.txt") as f:
        lines = [line.strip() for line in f]

    blocks = parse_blocks(lines)
    parameters = [extract_parameters(b) for b in blocks]

    digits = solve_monad(parameters)
    print("".join(str(d) for d in digits))

92915979999498


In [2]:
def parse_blocks(lines):
    # Each block is 18 lines, starting with "inp w"
    blocks = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("inp w"):
            blocks.append(lines[i:i+18])
            i += 18
        else:
            i += 1
    return blocks

def extract_parameters(block):
    # Given a single 18-line block, extract:
    # div z X  -> X is line 4
    # add x X  -> X is line 5 after "mod x 26"
    # add y Y  -> Y is line 15
    divz = int(block[4].split()[-1])
    checkx = int(block[5].split()[-1])
    addy = int(block[15].split()[-1])
    return divz, checkx, addy

def solve_monad(parameters):
    stack = []
    relations = [None] * 14

    for i, (divz, checkx, addy) in enumerate(parameters):
        if divz == 1:
            # push
            stack.append((i, addy))
        else:
            # pop
            j, prev_addy = stack.pop()
            # relation: digit[i] = digit[j] + (prev_addy + checkx)
            relations[i] = (j, prev_addy + checkx)

    digits = [0]*14

    # For smallest number, we try to assign digits starting from the lowest possible values.
    for i, rel in enumerate(relations):
        if rel is None:
            continue
        j, offset = rel

        # We have: digit[i] = digit[j] + offset
        # We want to minimize the resulting 14-digit number, meaning we should try to assign 
        # the earliest digits as small as possible.
        #
        # If offset > 0:
        #   digit[i] = digit[j] + offset
        #   To get minimal overall number, start with digit[j] as small as possible (1),
        #   then digit[i] = 1 + offset. If that doesn't fit (<=9), we adjust digit[j].
        #
        # If offset <= 0:
        #   To minimize, we try digit[i] as small as possible (1), so:
        #   1 = digit[j] + offset => digit[j] = 1 - offset
        #   If that doesn't fit in [1..9], we adjust accordingly.

        if offset > 0:
            # offset > 0
            # Try digit[j] = 1 => digit[i] = 1+offset
            dj = 1
            di = 1 + offset
            if di > 9:
                # Increase dj until di <= 9
                # di = dj + offset => dj = di - offset
                # For di=9, dj=9-offset
                dj = 9 - offset
                di = 9
                if dj < 1:
                    # If still doesn't fit, try a balanced approach:
                    # We want 1<=di,dj<=9 and di-dj=offset
                    found = False
                    for candidate_dj in range(1,10):
                        candidate_di = candidate_dj + offset
                        if 1<=candidate_di<=9:
                            dj = candidate_dj
                            di = candidate_di
                            found = True
                            break
                    if not found:
                        raise ValueError("No valid assignment found for offset > 0.")
        else:
            # offset <= 0
            # di = dj + offset
            # Try to minimize, start with di=1 => dj=1 - offset
            di = 1
            dj = 1 - offset
            if dj < 1:
                # dj too small, increase di
                # For dj=1, di=1+offset
                dj = 1
                di = 1 + offset
                if di < 1:
                    # Try a balanced approach for offset <=0
                    found = False
                    for candidate_di in range(1,10):
                        candidate_dj = candidate_di - offset
                        if 1<=candidate_dj<=9:
                            di = candidate_di
                            dj = candidate_dj
                            found = True
                            break
                    if not found:
                        raise ValueError("No valid assignment found for offset <= 0.")
            if di > 9:
                # If di > 9, we must lower di by adjusting dj
                # Try di=9 => dj=9 - offset
                di = 9
                dj = 9 - offset
                if dj < 1 or dj > 9:
                    # Try balanced again
                    found = False
                    for candidate_di in range(1,10):
                        candidate_dj = candidate_di - offset
                        if 1<=candidate_dj<=9:
                            di = candidate_di
                            dj = candidate_dj
                            found = True
                            break
                    if not found:
                        raise ValueError("No valid assignment found after adjustment.")

        # Check final ranges
        if not (1 <= di <= 9 and 1 <= dj <= 9):
            # If still invalid, try brute force approach:
            found = False
            for candidate_dj in range(1,10):
                candidate_di = candidate_dj + offset
                if 1 <= candidate_di <= 9:
                    dj = candidate_dj
                    di = candidate_di
                    found = True
                    break
            if not found:
                raise ValueError("No valid digits found for relation after all attempts.")

        digits[i] = di
        digits[j] = dj

    # For digits not yet assigned (from push blocks that never got paired),
    # set them to 1 to get the smallest number.
    for idx, d in enumerate(digits):
        if d == 0:
            digits[idx] = 1

    return digits

if __name__ == "__main__":
    with open("input.txt") as f:
        lines = [line.strip() for line in f]

    blocks = parse_blocks(lines)
    parameters = [extract_parameters(b) for b in blocks]

    digits = solve_monad(parameters)
    print("".join(str(d) for d in digits))

21611513911181
